# ETL Pipeline

## Importing Libraries

In [34]:
import pandas as pd # Data Transformation
from datetime import datetime 
import os
from subprocess import call
from dotenv import load_dotenv
from sqlalchemy import create_engine, text, INT, VARCHAR, DATE, TIMESTAMP, DECIMAL
from sqlalchemy.exc import SQLAlchemyError

## Setting Up PostgreSQL Connection

### Loading Environmental Variables

In [35]:
# Load environment variables from .env file
load_dotenv()

# Retrieve individual components from environment variables
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
host = os.getenv('POSTGRES_HOST')
port = os.getenv('POSTGRES_PORT')
db_name = os.getenv('POSTGRES_DB')

# Ensure the connection URI is retrieved successfully
if not all([user, password, host, db_name]):
    raise ValueError("One or more environment variables for the database connection are not set")

# Construct the connection URI
connection_uri = f"postgresql://{user}:{password}@{host}:{port}/{db_name}"

# Ensure the connection URI is retrieved successfully
if connection_uri is None:
    raise ValueError("DATABASE_URL environment variable is not set")

## Creating Schemas, Tables, and Views in PostgreSQL

### Creating a PostgreSQL Connection Engine with SQLAlchemy

In [36]:
# Define function to create an SQLAlchemy engine
def create_db_engine(connection_uri: str):
    """
    Create and return a SQLAlchemy engine based on the provided connection URI.

    Args:
        connection_uri (str): The connection URI for the database.

    Returns:
        Engine: A SQLAlchemy engine connected to the specified database.
    """
    try:
        db_engine = create_engine(connection_uri)
        print("Database engine created successfully.")
    except SQLAlchemyError as e:
        print(f"Error occurred while creating the database engine: {str(e)}")
        return None
    # Log or handle the error as needed
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        return None
    
    return db_engine

### Executing SQL Scripts Against PostgreSQL (Schemas, Tables, Views)

In [37]:
# Function to run SQL script using shell command
# I had to pass the env parameters explicitly  to the subprocess.call() -> (PGPASSWORD, PGUSER, PGHOST, PGPORT, PGDATABASE)
# This avoided Jupyter Notebook asking for password. 
def run_sql_script(script_name):
    script_path = f"/workspace/sql_scripts/{script_name}"
    command = f"psql -U {user} -d {db_name} -h {host} -p {port} -f {script_path}"
    return call(command, shell=True, env={
                                        'PGPASSWORD': password,
                                        'PGUSER': user,
                                        'PGHOST': host,
                                        'PGPORT': port,
                                        'PGDATABASE': db_name
    })

### Checking if Schemas Exist in PostgreSQL

In [38]:
# Function to check schema existence
def check_schema_existence(connection_uri, schema_names):
    try:
        db_engine = create_db_engine(connection_uri)
        if db_engine is None:
            print("Failed to create the database engine.")
            return
        
        with db_engine.connect() as connection:
            print("--- Checking if Schemas exist in the database ---")
            for schema_name in schema_names:
                result = connection.execute(
                    text("SELECT schema_name FROM information_schema.schemata WHERE schema_name = :schema"),
                    {"schema": schema_name}
                )
                schema_exists = result.fetchone() is not None
                if schema_exists:
                    print(f"Schema '{schema_name}' exists in the database.")
                else:
                    print(f"Schema '{schema_name}' does not exist in the database.")
            print("----- End of Schema Checking -----")
    
    except SQLAlchemyError as e:
        print(f"Error occurred while connecting to the database or executing query: {str(e)}")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

### Checking if Tables Exist in PostgreSQL

In [39]:
# Function to check table existence
def check_table_existence(connection_uri, schema_name, table_names):
    try:
        db_engine = create_db_engine(connection_uri)
        if db_engine is None:
            print("Failed to create the database engine.")
            return
        
        with db_engine.connect() as connection:
            print("--- Checking if Tables exist ---")
            for table_name in table_names:
                result = connection.execute(
                    text("SELECT table_name FROM information_schema.tables WHERE table_schema = :schema AND table_name = :table"),
                    {"schema": schema_name, "table": table_name}
                )
                table_exists = result.fetchone() is not None
                if table_exists:
                    print(f"Table '{table_name}' exists in schema '{schema_name}'.")
                else:
                    print(f"Table '{table_name}' does not exist in schema '{schema_name}'.")
            print("----- End of Checking Tables -----")
    
    except SQLAlchemyError as e:
        print(f"Error occurred while connecting to the database or executing query: {str(e)}")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

### Mapping columns between Bronze and Silver

In [40]:
def map_bronze_columns(table_name):
    """
    Maps column names from CSV dataframes to corresponding column names in bronze tables.

    Args:
        table_name (str): Name of the table for which column mapping is required.

    Returns:
        dict: A dictionary mapping column names in CSV dataframes to their corresponding column names in bronze tables.
    """
    if table_name == 'customers':
        return {
            'CustomerID': 'customer_id',
            'Name': 'name',
            'Age': 'age',
            'Gender': 'gender',
            'SignupDate': 'signup_date',
            'extracted_at': 'extracted_at',
            'inserted_at': 'inserted_at'
        }
    elif table_name == 'subscriptions':
        return {
            'SubscriptionID': 'subscription_id',
            'CustomerID': 'customer_id',
            'StartDate': 'start_date',
            'EndDate': 'end_date',
            'Type': 'type',
            'Status': 'status',
            'extracted_at': 'extracted_at',
            'inserted_at': 'inserted_at'
        }
    elif table_name == 'product_usage':
        return {
            'UsageID': 'usage_id',
            'CustomerID': 'customer_id',
            'DateID': 'date_id',
            'ProductID': 'product_id',
            'NumLogins': 'num_logins',
            'Amount': 'amount',
            'extracted_at': 'extracted_at',
            'inserted_at': 'inserted_at'
        }
    elif table_name == 'support_interactions':
        return {
            'InteractionID': 'interaction_id',
            'CustomerID': 'customer_id',
            'DateID': 'date_id',
            'IssueType': 'issue_type',
            'ResolutionTime': 'resolution_time',
            'extracted_at': 'extracted_at',
            'inserted_at': 'inserted_at'
        }
    elif table_name == 'dates':
        return {
            'DateID': 'date_id',
            'Date': 'date',
            'Week': 'week',
            'Month': 'month',
            'Quarter': 'quarter',
            'Year': 'year',
            'extracted_at': 'extracted_at',
            'inserted_at': 'inserted_at'
        }
    elif table_name == 'products':
        return {
            'ProductID': 'product_id',
            'ProductName': 'product_name',
            'Category': 'category',
            'Price': 'price',
            'extracted_at': 'extracted_at',
            'inserted_at': 'inserted_at'
        }
    else:
        raise ValueError(f"Table '{table_name}' not found in the bronze layer.")

### Getting Tables Column Names in the Silver Layer

In [41]:
def get_schema_table_columns(connection_uri, schema_name, tables_in_schema):
    """
    Fetches column names for a set of tables in a specified schema from a database.

    Args:
        connection_uri (str): The database connection URI.
        schema_name (str): The schema name where the tables are located.
        tables_in_silver (list of str): A list of table names for which the column names are to be fetched.

    Returns:
        dict: A dictionary where the keys are table names and the values are lists of column names for each table.
    """
    columns_dict = {}
    try:
        engine = create_db_engine(connection_uri)
        if engine is None:
            print("Failed to create the database engine.")
        
        with engine.connect() as connection:
            for table_name in tables_in_schema:
                query = text(f"""
                    SELECT column_name 
                    FROM information_schema.columns 
                    WHERE table_schema = '{schema_name}' 
                    AND table_name = '{table_name}';
                """)
                result = connection.execute(query)
                columns = [row[0] for row in result]  # Extract the first element (column_name) and create a list columns of column_names
                columns_dict[table_name] = columns # Fill the columns_dict with keys (table_name) and values (list of column names) 

    except Exception as e:
        print(f"Error occurred while fetching view columns: {str(e)}")

    return columns_dict

### Mapping Bronze Tables Data Types

In [42]:
def get_bronze_table_data_types():
    """
    Returns a dictionary with data types for columns in bronze tables.
    """
    bronze_data_types = {
        'customers': {
            'CustomerID': INT,
            'Name': VARCHAR(100),
            'Age': INT,
            'Gender': VARCHAR(10),
            'SignupDate': DATE,
            'extracted_at': TIMESTAMP,
            'inserted_at': TIMESTAMP
        },
        'dates': {
            'DateID': INT,
            'Date': DATE,
            'Week': INT,
            'Month': INT,
            'Quarter': INT,
            'Year': INT,
            'extracted_at': TIMESTAMP,
            'inserted_at': TIMESTAMP
        },
        'product_usage': {
            'UsageID': INT,
            'CustomerID': INT,
            'DateID': INT,
            'ProductID': INT,
            'NumLogins': INT,
            'Amount': DECIMAL(10, 2),
            'extracted_at': TIMESTAMP,
            'inserted_at': TIMESTAMP
        },
        'products': {
            'ProductID': INT,
            'ProductName': VARCHAR(100),
            'Category': VARCHAR(50),
            'Price': DECIMAL(10, 2),
            'extracted_at': TIMESTAMP,
            'inserted_at': TIMESTAMP
        },
        'subscriptions': {
            'SubscriptionID': INT,
            'CustomerID': INT,
            'StartDate': DATE,
            'EndDate': DATE,
            'Type': VARCHAR(50),
            'Status': VARCHAR(50),
            'extracted_at': TIMESTAMP,
            'inserted_at': TIMESTAMP
        },
        'support_interactions': {
            'InteractionID': INT,
            'CustomerID': INT,
            'DateID': INT,
            'IssueType': VARCHAR(100),
            'ResolutionTime': INT,
            'extracted_at': TIMESTAMP,
            'inserted_at': TIMESTAMP
        }
    }
    return bronze_data_types

### Mapping Silver Table Data Types

In [43]:
def get_silver_table_data_types():
    """
    Returns a dictionary with data types for columns in silver tables.
    """
    silver_data_types = {
        'dim_customers': {
            'customer_id': INT,
            'name': VARCHAR(100),
            'age': INT,
            'gender': VARCHAR(10),
            'signup_date': DATE,
            'extracted_at': TIMESTAMP,
            'inserted_at': TIMESTAMP
        },
        'dim_dates': {
            'date_id': INT,
            'date': DATE,
            'week': INT,
            'month': INT,
            'quarter': INT,
            'year': INT,
            'extracted_at': TIMESTAMP,
            'inserted_at': TIMESTAMP
        },
        'fact_product_usage': {
            'usage_id': INT,
            'customer_id': INT,
            'date_id': INT,
            'product_id': INT,
            'num_logins': INT,
            'amount': DECIMAL(10, 2),
            'extracted_at': TIMESTAMP,
            'inserted_at': TIMESTAMP
        },
        'dim_products': {
            'product_id': INT,
            'product_name': VARCHAR(100),
            'category': VARCHAR(50),
            'price': DECIMAL(10, 2),
            'extracted_at': TIMESTAMP,
            'inserted_at': TIMESTAMP
        },
        'fact_subscriptions': {
            'subscription_id': INT,
            'customer_id': INT,
            'start_date': DATE,
            'end_date': DATE,
            'type': VARCHAR(50),
            'status': VARCHAR(50),
            'extracted_at': TIMESTAMP,
            'inserted_at': TIMESTAMP
        },
        'fact_support_interactions': {
            'interaction_id': INT,
            'customer_id': INT,
            'date_id': INT,
            'issue_type': VARCHAR(100),
            'resolution_time': INT,
            'extracted_at': TIMESTAMP,
            'inserted_at': TIMESTAMP
        }
    }
    return silver_data_types

### Mapping Gold Table Data Types

## Extract

In [44]:
# Define function to extract data from CSV files
def extract(csv_folder_path):
    """
    Extract data from all CSV files in a folder, one by one.
    
    Args:
    - csv_folder_path (str): Path to the folder containing CSV files.
    
    Returns:
    - dict: A dictionary where keys are table names and values are DataFrames containing data from each CSV file.
    """
    # Test if a folder path exists
    if not os.path.exists(csv_folder_path):
        print(f"Folder '{csv_folder_path}' does not exist.")
        return None
    
    # Create a list of CSV files in the designated folder
    csv_files = [f for f in os.listdir(csv_folder_path) if f.endswith('.csv')]
    if not csv_files:
        print(f"No CSV files found in folder '{csv_folder_path}'.")
        return None
    
    # Create a dictionary where keys are table names and values are DataFrames containing data from each CSV file
    # This allows us to iterate over all the tables and perform specific transformations in the transform_raw() function  
    data_frames = {}

    # Iterating over each CSV file in the folder
    for csv_file in csv_files:
        # Separate the file name from the extension and store it
        table_name = os.path.splitext(csv_file)[0]  # Assuming table name is CSV filename without extension
        # Join CSV folder path with the CSV file name, inserting '/' as needed
        file_path = os.path.join(csv_folder_path, csv_file)
        try:
            df = pd.read_csv(file_path)
            print(f"-> CSV file '{csv_file}' loaded successfully.")
            
            # Add 'extracted_at' column with current timestamp
            df['extracted_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
            # Store the CSV in DataFrame format as a value of the dictionary's key
            data_frames[table_name] = df
        except Exception as e:
            print(f"Error reading CSV file '{csv_file}': {str(e)}")
            data_frames[table_name] = None
    
    # Return the dictionary
    return data_frames

## Transform

**[Based on the CSV Files]**

1) Date Columns were converted to conform to PostgreSQL syntax.
2) Column Names were renamed (The tables in the Silver Layer will contain renamed columns. See below).

*Customers*
* SignupDate: will be converted to datetime (to be used in PostgreSQL).

*Dates*
* DateID : will be converted from boolean to integer (to be used in PostgreSQL).

*Product Usage*
* Date column not available.

*Subscriptions*
* StartDate,EndDate: will be converted to datetime (to be used in PostgreSQL).
* Status: will be converted from boolean to integer (to be used in PostgreSQL).

*Support Interactions*
* Date column not available.

In [45]:
def transform_raw(dfs, date_columns_map):
    """
    Transform multiple raw DataFrames extracted from CSV files.
    Perform cleaning procedures: Convert specified date columns to pandas datetime and 
    rename columns using map_bronze_columns function.

    Args:
        data_frames (dict): A dictionary where keys are table names and values are DataFrames 
                            containing raw data extracted from CSV files.
        date_columns_map (dict): A dictionary where keys are table names and values are 
                                 column names to convert to pandas datetime.

    Returns:
        dict: A dictionary where keys are table names and values are cleaned DataFrames 
              with the specified transformations applied.
    """
    # Create a dictionary where keys are table names and values are the clean DataFrames after performing specific transformations.
    cleaned_data_frames = {}

    # Iterate over all tables. If table is not in the dict returned by the extract() function, then it is skipped for transformation.
    # df contains the actual data (in the DataFrame format) for each table
    for table_name, df in dfs.items():
    
        try:
            if table_name in date_columns_map:
                # We specify the date columns in each table to perform transformations.
                # If a value of a key of date_columns_map is a single date column:
                #   Ex: date_columns == 'SignupDate', a string.
                # If a value of a key of date_columns_map is a list of date columns:
                #   Ex: date_columns == ['StartDate', 'EndDate'], a list.
                date_columns = date_columns_map[table_name]  # accessing the value of the key 'table_name'
            
                # Note: the transform_raw() can receive a list of date columns, so we need to ensure the date_columns variable
                # is always treated as a list, even if a single date column name is provided.
                if not isinstance(date_columns, list):
                    date_columns = [date_columns]

                # Iterate over a potential list of columns (either single or multiple), one by one making the transformation.
                for date_column in date_columns:
                    # Check if the date column exists in the DataFrames that correspond to the data of each table. 
                    if date_column not in df.columns:
                        raise ValueError(f"Column '{date_column}' does not exist in the DataFrame for table '{table_name}'.")
                    
                    # Format the date column to 'YYYY-MM-DD' format
                    df[date_column] = pd.to_datetime(df[date_column]).dt.strftime('%Y-%m-%d')
                    print(f"Successfully converted column '{date_column}' to 'YYYY-MM-DD' format for table '{table_name}'.")
                    print(f"Data type after conversion: {df[date_column].dtype}")

            # Rename columns using the map_bronze_columns function
            column_mapping = map_bronze_columns(table_name)
            df.rename(columns=column_mapping, inplace=True)
            print(f"Columns renamed for table '{table_name}'.")

            # Add the cleaned DataFrame to the cleaned_data_frames dictionary
            cleaned_data_frames[table_name] = df

        except ValueError as ve:
            print(ve)
            # Indicates that an error occurred during the processing of the DataFrame for table_name and it
            # sets to None to signify that the data transformation or cleaning for that table was unsuccessful.
            cleaned_data_frames[table_name] = None
        except Exception as e:
            print(f"An error occurred when converting the date for table '{table_name}': {e}")
            cleaned_data_frames[table_name] = None
            
    # Returns a clean DataFrame when dates have been treated adn columns have been renamed.
    return cleaned_data_frames

## Data Ingestion into the Bronze Layer

### Load

In [46]:
def ingest_csv_to_bronze(dfs, connection_uri, bronze_schema_name):
    """
    Extract, transform, and ingest CSV data into the Bronze layer of a PostgreSQL database.

    Args:
        csv_folder_path (str): Path to the folder containing CSV files.
        connection_uri (str): Connection URI for the PostgreSQL database.
        schema_name (str): Name of the schema in which tables exist or will be created.
        date_columns_map (dict): A dictionary where keys are table names and values are columns to convert to pandas datetime.

    Returns:
        dict: A dictionary where keys are table names and values are DataFrames with the transformed data.
    """

    print("----- Ingesting Data Into Bronze Layer. -----")

    try:
        # Create the database engine
        engine = create_engine(connection_uri)

        # Verify connection and schema existence
        with engine.connect() as connection:
            # Set the search path to the specified schema
            set_search_path_query = text(f"SET search_path TO {bronze_schema_name};")
            connection.execute(set_search_path_query)
            print(f"Search path set to schema '{bronze_schema_name}'.")

            # Iterate over transformed DataFrames and ingest data into the database
            print("-- to_sql() Ingestion Procedure in Bronze. --")
            for table_name, cleaned_data_df in dfs.items():
                if cleaned_data_df is None:
                    print(f"Skipping ingestion for table '{table_name}' due to previous errors.")
                    continue

                print(f"Ingesting data into {bronze_schema_name}.{table_name}...")

                # Add 'inserted_at' timestamp columns
                cleaned_data_df['inserted_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

                # Get data types for the table from the dictionary
                bronze_data_types = get_bronze_table_data_types()
                data_type_dict = bronze_data_types.get(table_name)

                if data_type_dict is None:
                    raise ValueError(f"Data types not found for table '{table_name}' in bronze layer.")

                # Ingest data into the specified schema and table with specified data types
                cleaned_data_df.to_sql(table_name, engine, schema=bronze_schema_name, if_exists='replace', index=False, dtype=data_type_dict)

                print(f"-> CSV data ingested successfully into {bronze_schema_name}.{table_name}.")

        print("----- END OF DATA INGESTION INTO BRONZE -----")

    except FileNotFoundError:
        print("Ingest Function: Error - CSV file not found.")
    except SQLAlchemyError as e:
        print(f"Error occurred while connecting to the database or ingesting data: {str(e)}")
    except ValueError as ve:
        print(f"ValueError: {str(ve)}")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

## Data Integration and Transformation into the Silver Layer

In [47]:
def ingest_bronze_to_silver(transformed_data_dfs, connection_uri, silver_schema_name, tables_in_silver, create_silver_tables_script_path):
    print("----- Ingesting Data Into Silver Layer. -----")

    try:
        # Create database engine
        engine = create_engine(connection_uri)
        if engine is None:
            print("Failed to create the database engine.")
            return

        print("Database engine created successfully.")

        # 6) (Silver) Run create_silver_tables.sql
        print("----- Creating TABLES in Silver -----")
        result = run_sql_script(create_silver_tables_script_path)
        if result == 0:
            print("SQL script executed successfully. Tables were created in the Siver Layer.")
            # Check if schemas exist in the database
            check_table_existence(connection_uri, silver_schema_name, tables_in_silver)
        else:
            print("Error executing SQL script.")

        # Verify connection and schema existence
        with engine.connect() as connection:
            # Check if the silver schema exists
            schema_exists_query = text(f"""
                SELECT schema_name 
                FROM information_schema.schemata 
                WHERE schema_name = :schema_name
            """)
            result = connection.execute(schema_exists_query, {"schema_name": silver_schema_name})
            if result.fetchone() is None:
                raise ValueError(f"Schema '{silver_schema_name}' does not exist in the database.")
            
            # Set the search path to the silver schema
            connection.execute(text(f"SET search_path TO {silver_schema_name};"))
            print(f"Search path set to schema '{silver_schema_name}'.")

            # Iterate over transformed DataFrames and ingest data into the database tables
            for silver_table_name, (temp, cleaned_data_df) in zip(tables_in_silver, transformed_data_dfs.items()):
                if cleaned_data_df is None:
                    print(f"Skipping ingestion for table '{silver_table_name}' due to previous errors.")
                    continue

                # Add 'inserted_at' timestamp columns
                cleaned_data_df['inserted_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

                # Get data types for the table from the dictionary
                silver_data_types = get_silver_table_data_types()
                data_type_dict = silver_data_types.get(silver_table_name)

                if data_type_dict is None:
                    raise ValueError(f"Data types not found for table '{silver_table_name}' in silver layer.")

                # Ingest data into the specified schema and table with specified data types
                cleaned_data_df.to_sql(silver_table_name, engine, schema=silver_schema_name, if_exists='replace', index=False, dtype=data_type_dict)

                print(f"-> Data data ingested successfully into {silver_schema_name}.{silver_table_name}.")
    
        print("----- END OF DATA INGESTION IN SILVER -----")
    
    except SQLAlchemyError as e:
        print(f"SQLAlchemyError occurred while connecting to the database or ingesting data: {str(e)}")
    except ValueError as ve:
        print(f"ValueError: {str(ve)}")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

# def ingest_bronze_to_silver(connection_uri, bronze_schema_name, silver_schema_name, transformed_data_dfs, tables_in_silver):
#     print("----- Ingesting Data Into Silver Layer. -----")

#     try:
#         # Create database engine
#         engine = create_engine(connection_uri)
#         if engine is None:
#             print("Failed to create the database engine.")
#             return

#         print("Database engine created successfully.")

#         # Verify connection and schema existence
#         with engine.connect() as connection:
#             # Check if the silver schema exists
#             schema_exists_query = text(f"""
#                 SELECT schema_name 
#                 FROM information_schema.schemata 
#                 WHERE schema_name = :schema_name
#             """)
#             result = connection.execute(schema_exists_query, {"schema_name": silver_schema_name})
#             if result.fetchone() is None:
#                 raise ValueError(f"Schema '{silver_schema_name}' does not exist in the database.")
            
#             # Set the search path to the silver schema
#             connection.execute(text(f"SET search_path TO {silver_schema_name};"))
#             print(f"Search path set to schema '{silver_schema_name}'.")

#             # Iterate over transformed DataFrames and ingest data into the database tables
#             # print("-- Ingestion Procedure in Silver. --")
#             # for silver_table_name, (bronze_table_name, cleaned_data_df) in zip(tables_in_silver, transformed_data_dfs.items()):
#             #     if cleaned_data_df is None:
#             #         print(f"Skipping ingestion for table '{silver_table_name}' due to previous errors.")
#             #         continue
#             print("-- to_sql() Ingestion Procedure in Silver. --")
#             for table_name, cleaned_data_df in transformed_data_dfs.items():
#                 if cleaned_data_df is None:
#                     print(f"Skipping ingestion for table '{table_name}' due to previous errors.")
#                     continue

#                 # Add 'extracted_at' and 'inserted_at' timestamp columns for the ingestion into Silver
#                 cleaned_data_df['extracted_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#                 cleaned_data_df['inserted_at']  = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

#                 # Ingest data into the specified schema and table using to_sql
#                 cleaned_data_df.to_sql(table_name, engine, schema=silver_schema_name, if_exists='replace', index=False, dtype=get_silver_table_data_types().get(silver_table_name))

#                 print(f"-> Data ingested successfully into {silver_schema_name}.{silver_table_name}.")
    
#         print("----- END OF DATA TRANSFORMATION IN SILVER -----")
#         # return aggregated_data_dfs
    
#     except SQLAlchemyError as e:
#         print(f"SQLAlchemyError occurred while connecting to the database or ingesting data: {str(e)}")
#     except ValueError as ve:
#         print(f"ValueError: {str(ve)}")
#     except Exception as e:
#         print(f"An unexpected error occurred: {str(e)}")

## Data Aggregation and Feature Engineering into the Gold Layer

In [48]:
# def ingest_silver_to_gold(connection_uri, silver_schema_name, gold_schema_name, tables_in_gold, create_gold_tables_script_path):
#     print("----- Ingesting Data Into Gold Layer. -----")

#     try:
#         # Create database engine
#         engine = create_engine(connection_uri)
#         if engine is None:
#             print("Failed to create the database engine.")
#             return

#         print("Database engine created successfully.")
        
#         # Verify connection and schema existence
#         with engine.connect() as connection:
#             # Check if the gold schema exists
#             schema_exists_query = text(f"""
#                 SELECT schema_name 
#                 FROM information_schema.schemata 
#                 WHERE schema_name = :schema_name
#             """)
#             result = connection.execute(schema_exists_query, {"schema_name": gold_schema_name})
#             if result.fetchone() is None:
#                 raise ValueError(f"Schema '{gold_schema_name}' does not exist in the database.")
            
#             # Set the search path to the gold schema
#             connection.execute(text(f"SET search_path TO {gold_schema_name};"))
#             print(f"Search path set to schema '{gold_schema_name}'.")

#             # Extract data using the SELECT queries embedded in the Gold SQL script
#             print("-- Extracting Data for Ingestion into Gold. --")
#             aggregated_data_dfs = {}
#             for table in tables_in_gold:
#                 query = text(f"SELECT * FROM {gold_schema_name}.{table}")
#                 df = pd.read_sql(query, connection)
#                 aggregated_data_dfs[table] = df
#                 print(f"Data extracted for table '{table}'")

#             # Iterate over aggregated DataFrames and ingest data into the database tables
#             print("-- Ingestion Procedure in Gold. --")
#             for gold_table_name, aggregated_data_df in aggregated_data_dfs.items():
#                 if aggregated_data_df is None:
#                     print(f"Skipping ingestion for table '{gold_table_name}' due to previous errors.")
#                     continue

#                 # Add 'extracted_at' and 'inserted_at' timestamp columns for the ingestion into Gold
#                 aggregated_data_df['extracted_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#                 aggregated_data_df['inserted_at']  = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

#                 # Ingest data into the specified schema and table using to_sql
#                 aggregated_data_df.to_sql(gold_table_name, engine, schema=gold_schema_name, if_exists='replace', index=False)

#                 print(f"-> Data ingested successfully into {gold_schema_name}.{gold_table_name}.")

#         print("----- END OF DATA INGESTION IN GOLD -----")
    
#     except SQLAlchemyError as e:
#         print(f"SQLAlchemyError occurred while connecting to the database or ingesting data: {str(e)}")
#     except ValueError as ve:
#         print(f"ValueError: {str(ve)}")
#     except Exception as e:
#         print(f"An unexpected error occurred: {str(e)}")


## Executing

In [49]:
# Ingestion Parameters for Bronze, Silver, and Gold Layers
csv_folder_path = '/workspace/data/raw'
schema_names = ['bronze', 'silver', 'gold']
bronze_schema = 'bronze'
silver_schema = 'silver'
gold_schema   = 'gold'

create_schemas_script_path       = 'schemas/create_schemas.sql'
create_bronze_tables_script_path = 'bronze/create_bronze_tables.sql'
create_silver_tables_script_path = 'silver/create_silver_tables.sql'
create_gold_tables_script_path   = 'gold/create_gold_tables.sql'

# Note, do not alter the order of the table names in silver, or it will not be ingested correctly
tables_in_bronze = ['customers', 'dates', 'product_usage', 'products', 'subscriptions', 'support_interactions']
tables_in_silver = ['dim_customers', 'dim_dates', 'dim_products', 'fact_product_usage', 'fact_subscriptions', 'fact_support_interactions']
tables_in_gold   = ['churn_customers', 'product_usage_features', 'support_interaction_features', 'churn_features']

# Define date columns for each table in Bronze to perform specific transformations to conform to PostgreSQL syntax.
date_columns_map = {            
    # Date columns should be follow the CSV files column names
    'customers': 'SignupDate',
    'dates': 'Date',
    'subscriptions': ['StartDate', 'EndDate'],
}

# Execute functions (please, respect the order)
    # 1) Run create_schemas.sql
    # 2) (Bronze) Calling the Extract Function for all CSV files
    # 3) (Bronze) Data Ingestion with Minor Transformation
    # 4) (Silver) Calling the Transformation Function for the DataFramese based on the CSV files
    # 5) (Silver) Data Integration and Transformation
    # 6) (Gold) Data Aggregation and Feature Engineering - Run create_gold_tables.sql

# 1) Run create_schemas.sql 
print("----- Creating SCHEMAS in PostgreSQL -----")
result = run_sql_script(create_schemas_script_path)
if result == 0:
    print("SQL script executed successfully. Schemas were created.")
    # Check if schemas exist in the database
    check_schema_existence(connection_uri, schema_names)
else:
    print("Error executing SQL script.")

# 2) (Bronze) Calling the Extract Function for all CSV files
print(" -- Extract Function. --")
raw_data_dfs = extract(csv_folder_path)
if raw_data_dfs is None:
    print("Extraction failed.")

# 3) (Bronze) Data Ingestion with Minor Transformation
# transformed_data_dfs = ingest_csv_to_bronze(csv_folder_path, connection_uri, bronze_schema, date_columns_map)
ingest_csv_to_bronze(raw_data_dfs, connection_uri, bronze_schema)

# TEST COLUMN NAMES BEFORE INGESTION IN BRONZE
print("--- PRINTING COLUMN NAMES AFTER INGESTING IN BRONZE---")
bronze_column_names_before_ingestion = get_schema_table_columns(connection_uri, bronze_schema, tables_in_bronze)
print(bronze_column_names_before_ingestion.items())

# 4) Calling the Transformation Function for the DataFramese based on the CSV files
print("-- Transformation Function. --")
transformed_data_dfs = transform_raw(raw_data_dfs, date_columns_map)
if transformed_data_dfs is None:
    print("Error occurred during transformation. Processing aborted.")

# 5) (Silver) Data Integration and Transformation
ingest_bronze_to_silver(transformed_data_dfs, connection_uri, silver_schema, tables_in_silver, create_silver_tables_script_path)

# TEST COLUMN NAMES AFTER INGESTION IN SILVER
print("--- PRINTING COLUMN NAMES AFTER INGESTION IN SILVER ---")
silver_columns_after_ingestion = get_schema_table_columns(connection_uri, silver_schema, tables_in_silver)
print(silver_columns_after_ingestion.items())

# 6) (Gold) Data Aggregation and Feature Engineering - Run create_gold_tables.sql
print("----- Creating TABLES in Gold -----")
result = run_sql_script(create_gold_tables_script_path)
if result == 0:
    print("SQL script executed successfully. Tables were created in the Gold Layer.")
    # Check if schemas exist in the database
    check_table_existence(connection_uri, gold_schema, tables_in_gold)
else:
    print("Error executing SQL script.")

# # 7) (Gold) Data Aggregation and Feature Engineering
# ingest_silver_to_gold(connection_uri, silver_schema, gold_schema, tables_in_gold, create_gold_tables_script_path)

----- Creating SCHEMAS in PostgreSQL -----
CREATE SCHEMA
CREATE SCHEMA
CREATE SCHEMA
SQL script executed successfully. Schemas were created.
Database engine created successfully.
--- Checking if Schemas exist in the database ---
Schema 'bronze' exists in the database.
Schema 'silver' exists in the database.
Schema 'gold' exists in the database.
----- End of Schema Checking -----
 -- Extract Function. --
-> CSV file 'customers.csv' loaded successfully.


-> CSV file 'dates.csv' loaded successfully.
-> CSV file 'products.csv' loaded successfully.
-> CSV file 'product_usage.csv' loaded successfully.
-> CSV file 'subscriptions.csv' loaded successfully.
-> CSV file 'support_interactions.csv' loaded successfully.
----- Ingesting Data Into Bronze Layer. -----
Search path set to schema 'bronze'.
-- to_sql() Ingestion Procedure in Bronze. --
Ingesting data into bronze.customers...
-> CSV data ingested successfully into bronze.customers.
Ingesting data into bronze.dates...
-> CSV data ingested successfully into bronze.dates.
Ingesting data into bronze.products...
-> CSV data ingested successfully into bronze.products.
Ingesting data into bronze.product_usage...
-> CSV data ingested successfully into bronze.product_usage.
Ingesting data into bronze.subscriptions...
-> CSV data ingested successfully into bronze.subscriptions.
Ingesting data into bronze.support_interactions...
-> CSV data ingested successfully into bronze.support_interactions.
---